Task 1: Third-order letter approximation model

First cell defines needed perameters such as links to files.
A dictionary to add each text to.
And Strings to define preamble and postable markers for removal.

In [59]:
files = [
    '/workspaces/emergingtechnologies/data/frankenstein.txt',
    '/workspaces/emergingtechnologies/data/bookplates.txt',
    '/workspaces/emergingtechnologies/data/spyglassmoutain.txt',
    '/workspaces/emergingtechnologies/data/theoldhouse.txt',
    '/workspaces/emergingtechnologies/data/windofdestiny.txt',
]

texts = {}

#preamble and postamble markers
start_marker = " ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

We then need to read each file, rather then doing this individually I've created a loop which adds each text to a dictionary.I've done it this way so I dont have to deal with indivdual texts and can simply loop over the dictionary when required to retrieve each text.

After this I used the .find feature within the same loop, to find the markers that I've predefined and extract all text between these markers using .strip, this removes the preamble and postamble. I then add this cleaned text to the dictionary with filename as the key as this allows me to print each text separatly to check each is cleaned and for future use.

In [60]:
#read each file and remove preamble and postamble
for file_path in files:
    with open(file_path, 'r') as f:
        text = f.read()

        #find the start and end markers
        start = text.find(start_marker)
        end = text.find(end_marker)

        #skip the start marker
        start += len(start_marker)
        #extract all text between the markers
        text = text[start:end].strip()

        #store the cleaned text in the dictionary with filename as the key
        filename = file_path.split('/')[-1]  # Get filename from path
        texts[filename] = text

I then create a new loop that iterates over each key-value pair in the texts dictionary. I had to add a .replace method to this to remove '/n' at it was showing up a number of times. I then define the charecters allowed in text to be fed to the model. Then changing my text to uppercase I clean the the text by removing any charecters not in the defined char set, before adding the cleaned text back to its key in the dictionary.

In [61]:
for filename, text in texts.items():
    #remove all newlines
    text = text.replace('\n', ' ')    
    text = text.upper()
    chars = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ. ')
    #remove all characters that are not in the set of allowed characters
    cleaned = ''.join(c for c in text if c in chars)
    #store the cleaned text back in the dictionary
    texts[filename] = cleaned

#print(texts['theoldhouse.txt'])

I then created a trigram model. Using the dictionary model and the same key value loop as before, I created a loop that iterates over the file provided in each loop, each trigram is extracted from the text. A if statement was required to remove empty values that were missed in the cleaning process. These trigrams are then added to the dictionary as a key with each occurance being counted and added to the value

In [62]:
model = {}

for filename, cleaned in texts.items():
  #iterate over the text and count the occurrences of each trigram
  for i in range(len(cleaned) - 2):
    #extract the trigram
    trigram = cleaned[i:i+3]
    if trigram != '' and trigram != '   ':
        #store the trigram in the model
      model[trigram] = model.get(trigram, 0) + 1

This is just a sorted example of the model.

In [65]:
#sorted(model.items(), key=lambda x: x[1], reverse=True)

Task 2: Third-order letter approximation generation

In [64]:
#trigram loop up dictionary for model
trigram_dict = {}

for trigram, count in model.items():
    if len(trigram) == 3:
        bigram = trigram[:2]
        next_char = trigram[2]
        if bigram not in trigram_dict:
            trigram_dict[bigram] = {}
        trigram_dict[bigram][next_char] = count

# Manually format and print the trigram dictionary
for bigram, next_chars in trigram_dict.items():
    print(f"Bigram '{bigram}':")
    for next_char, count in next_chars.items():
        print(f"  '{next_char}': {count}")
    print()  # Blank line for readability between bigrams

Bigram 'FR':
  'A': 252
  'O': 1166
  'E': 252
  'I': 294
  'U': 17
  '.': 3
  'Y': 3
  ' ': 59

Bigram 'RA':
  'N': 766
  'F': 13
  'C': 327
  'V': 460
  'T': 629
  'R': 191
  'D': 116
  'L': 401
  'G': 226
  'I': 432
  'B': 215
  'P': 137
  'S': 109
  'W': 117
  'M': 213
  'U': 12
  'O': 4
  ' ': 48
  'Y': 121
  'K': 8
  '.': 3
  'H': 2
  'Z': 31

Bigram 'AN':
  'K': 201
  'D': 9057
  'I': 231
  ' ': 3037
  'C': 810
  '.': 99
  'G': 631
  'N': 255
  'Y': 680
  'Q': 27
  'T': 1009
  'S': 350
  'O': 207
  'E': 105
  'X': 28
  'U': 41
  'W': 8
  'A': 73
  'L': 14
  'V': 7
  'F': 1
  'B': 6
  'J': 1
  'M': 7
  'H': 20

Bigram 'NK':
  'E': 80
  'I': 85
  ' ': 359
  'S': 73
  'N': 18
  'L': 37
  'H': 3
  '.': 15
  'F': 5
  'T': 1
  'M': 1
  'Y': 24
  'C': 3

Bigram 'KE':
  'N': 262
  'S': 100
  'D': 574
  ' ': 1091
  'E': 130
  'T': 142
  'Y': 40
  '.': 44
  'P': 37
  'F': 7
  'R': 110
  'B': 2
  'W': 3
  'L': 34
  'G': 2
  'I': 2
  'M': 5
  'A': 1

Bigram 'EN':
  'S': 371
  'T': 3283
  'G